## Python Module: Create a Lease with Multiple Resources Reservations

This notebook demonstrates how to use the OpenStack python API to create a lease containing multiple resources reservations. The example method creates reservations for one server, one network, and one floating IP. This example method is likely not useful on its own and should be used a model to develop your own multi-reservation lease requests.


Chameleon requires most resources to be reserved before they can be used. Reservations can not be created independently. Instead, all reservatons must be part of a *Lease*.  Leases are composed of a group of one or more reservations that, typically, include all reservations required for an experiment.  

#### Related Modules

- [Reserve a Server](./reserve_node_simple.ipynb)
- [Reserve a Network](./reserve_network_simple.ipynb)
- [Reserve a Floating IP](./reserve_floating_ip.ipynb)
- [Get Lease by Name](./get_lease_by_name.ipynb)
- [Delete Lease](./delete_lease.ipynb)

#### Import Library

```
import chi
from chi.reservation_api_examples import *
```

#### Arguments

Required:

- `lease_name`: The name of the targeted lease

Optional:

- None

#### Imported Methods used by Example(s) 

In [ ]:
from datetime import datetime, timedelta
from dateutil import tz

BLAZAR_TIME_FORMAT = "%Y-%m-%d %H:%M"

def add_node_reservation(reservation_list, count=1, node_type="compute_haswell"):
    reservation_list.append({
        "resource_type": "physical:host",
        "resource_properties": json.dumps(["==", "$node_type", node_type]),
        "hypervisor_properties": "",
        "min": count,
        "max": count
    })

def add_network_reservation(reservation_list, network_name="myNet",
                            network_description="", 
                            physical_network="physnet1"):
    reservation_list.append({
        "resource_type": "network",
        "network_name": network_name,
        "network_description": network_description,
        "resource_properties": json.dumps(["==", "$physical_network", physical_network]),
        "network_properties": ""
    })
    
def add_fip_reservation(reservation_list, count=1):
    #Get public network id (needed to reserve networks)
    public_network_id = get_public_network(chi.neutron())

    reservation_list.append({
        "resource_type": "virtual:floatingip",
        "network_id": public_network_id,
        "amount": count
    })    
    
    
def reserve_multiple_resources(lease_name):
    # Set start/end date for lease
    # Start one minute into future to avoid Blazar thinking lease is in past
    # due to rounding to closest minute.
    start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
    end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)
    
    # Build list of reservations (in this case there is only one reservation)
    reservation_list = []
    add_node_reservation(reservation_list, count=1, node_type="compute_haswell")
    add_network_reservation(reservation_list, network_name=lease_name+"Network")
    add_fip_reservation(reservation_list, count=1)
    
    # Create the lease
    lease = chi.blazar().lease.create(name=lease_name, 
                                start=start_date,
                                end=end_date,
                                reservations=reservation_list, events=[])

## Examples: 
**Important**: Examples below import API methods from chi library and does not use refernece code above!

#### Reserve Mulitpule Resources

In [4]:
import json
import os
import chi

from chi.reservation_api_examples import *

#Config with your project and site
chi.set('project_name', 'CH-816532') # Replace with your project name
chi.set('region_name', 'CHI@UC')     # Optional, defaults to 'CHI@UC'


# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
lease_name = username+'Lease'

#Get a network by name
lease = reserve_multiple_resources(lease_name)
    
#Get the lease by name
lease = get_lease(lease_name)
    
#Print the lease info
print(json.dumps(lease, indent=2))




{
  "status": "PENDING",
  "user_id": "f3a422ce5a4442e3a7a247ddfd98bfc3",
  "name": "pruthLease",
  "end_date": "2020-05-06T18:21:00.000000",
  "reservations": [
    {
      "status": "pending",
      "lease_id": "c67c6fcc-232d-442f-98af-e4d8f3eeb466",
      "resource_id": "63db6061-2605-4946-9a0d-29de21085d31",
      "created_at": "2020-05-05 18:21:22",
      "updated_at": "2020-05-05 18:21:24",
      "missing_resources": false,
      "id": "925b40d9-b59c-467c-bc3e-0836f1537b8f",
      "resource_type": "network",
      "resources_changed": false
    },
    {
      "status": "pending",
      "lease_id": "c67c6fcc-232d-442f-98af-e4d8f3eeb466",
      "resource_id": "1f1942be-8fdb-4d11-85c3-85b26bb3809e",
      "network_id": "44b38c44-2a42-4b6d-b129-6c8f1b2a1375",
      "created_at": "2020-05-05 18:21:24",
      "updated_at": "2020-05-05 18:21:25",
      "required_floatingips": [],
      "missing_resources": false,
      "amount": 1,
      "id": "bff972f8-69ab-4194-be88-da68940c057f",
   